In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# run this cell if running remotely

!pip install transformers==4.3.3
!pip install stop_words
!pip install symspellpy
!pip install language_detector
!pip install cached_property
!pip install sentencepiece
!pip install config
!pip install umap
!pip install sentence-transformers==0.4.1
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from google.colab import drive
drive.mount('/content/drive')

import tensorflow, sentence_transformers, transformers
tensorflow.__version__, sentence_transformers.__version__, transformers.__version__

root_path = "/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2"
import sys, os
import config

config.root_path = os.path.abspath(root_path)
sys.path.insert(0, config.root_path)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.8 MB/s 
     |████████████████████████████████| 880 kB 94.1 MB/s 
     |████████████████████████████████| 3.3 MB 78.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=519751a7524493a6bedf304e3e439d37498155835f71caa445eaeaf5601e23a6
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32910 sha256=d35b4b5d0d68ee6e16bfc0fad8dd5b9305253bde03bbc8a3bc14de1970ca6ee8
  Stored in directory: /root/.cache/pip/wheels/eb/03/0d/3bd31c983789aeb0b4d5e2ca48590288d9db1586cf5f225062
Successfully built stop-words
Looking in indexes: https://pypi.

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Mounted at /content/drive


In [ ]:
# run this cell if working locally
import sys, os
import config

config.root_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
sys.path.insert(0, config.root_path)

In [3]:
root_path = "/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2"

config.root_path = os.path.abspath(root_path)
sys.path.insert(0, config.root_path)
sys.path

['/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2',
 '/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2',
 '/content',
 '/env/python',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/usr/local/lib/python3.8/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.8/dist-packages/IPython/extensions',
 '/root/.ipython']

In [4]:
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from src.encoders.context_encoder_ldabert_2 import ContextEncoderSimple
from src.dataset.ldabert_2 import LDABERT2Dataset

from tensorflow.python import keras
import toml
import json
import pandas as pd
import numpy as np
from utils.experiments import get_experiments_json, get_experiments, save_results

Downloading:   0%|          | 0.00/630 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/409 [00:00<?, ?B/s]

## Experiment

In [5]:
experiments_config = get_experiments_json('ldabert2_simple_finetune_test')
experiments_config_df = pd.DataFrame.from_dict(experiments_config)
experiments_config_df

,bert_type,dataset_type,final_dropout,dense_neurons,max_sentence_length,gamma,pct_data,augment_pct,bert_trainable,epochs
0,ldabert,clinical,0.5,128,128,15,1,1,True,50
1,ldabert,wiki,0.5,128,128,15,1,1,True,50
2,ldabert,fiction,0.5,128,128,15,1,1,True,50


In [6]:
experiments_config_df.to_csv(r'../models/experiment.csv', header=None, index=None, sep=' ', mode='a')

In [7]:
def get_random_hash(k):
  import random, string
  x = ''.join(random.choices(string.ascii_letters + string.digits, k=k))
  return x

In [ ]:
# all
for experiment in experiments_config[2:3]:
    dataset_type = experiment['dataset_type']
    final_dropout = experiment['final_dropout']
    dense_neurons = experiment['dense_neurons']
    pct_data = experiment['pct_data']
    augment_pct = experiment['augment_pct']
    gamma = experiment['gamma']
    max_sentence_length = experiment['max_sentence_length']
    bert_trainable = experiment['bert_trainable']
    epochs = experiment['epochs']
    BATCH_SIZE = 16
    finetuning_epochs = 5 # finetune the system for 5 epochs before training with the frozen finetuned BERT weights
    print("params:", experiment)
    random_hash = get_random_hash(5)
    experiment['epochs'] = epochs

    for step in ["finetune","frozen"]:
    # for step in ["frozen"]:
        # init model
        print("initializing model...")
        model = ContextEncoderSimple(final_dropout=final_dropout,
                                dense_neurons=dense_neurons,
                              gamma=gamma,
                              max_sentence_length=max_sentence_length,
                               bert_trainable=bert_trainable)

        # print("number of params: ", sum([np.prod(keras.get_value(w).shape) for w in model.trainable_weights]))

        # init dataset
        print("initializing dataset...")
        dataset = LDABERT2Dataset(dataset_type=dataset_type,
                                pct_data=pct_data,
                              max_seq_length=max_sentence_length,
                                max_segment_length=300,
                                augment_pct=0,
                                split="train",
                            artificial_segments=False)

        # process dataset
        print("processing dataset...")
        sentences, tokenized_sentences, labels = dataset.process()


        vectors_filename = '{}_{}_as-{}_msl-{}.pkl'.format(dataset.pct_data, dataset.augment_pct, dataset.artificial_segments, dataset.max_segment_length)

        saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.get_saved_vectors("train", dataset.dataset_type, vectors_filename)

        if len(saved_vectors) == 0:
            saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.create_vectors("train", dataset.dataset_type, vectors_filename)

        left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(saved_tokenized_sentences)
        lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(saved_vectors)

        # init testing dataset
        print("initializing testing dataset...")
        testing_dataset = LDABERT2Dataset(dataset_type=dataset_type,
                                pct_data=0.25,
                              max_seq_length=512,
                                max_segment_length=300,
                                augment_pct=0,
                                split="test",
                            artificial_segments=False)

        # process testing dataset
        print("processing testing dataset...")
        testing_sentences, testing_tokenized_sentences, testing_labels = testing_dataset.process()

        vectors_filename = '{}_{}_testing_data.pkl'.format(testing_dataset.pct_data, testing_dataset.augment_pct)

        testing_saved_vectors, testing_saved_labels, testing_saved_sentences, testing_saved_tokenized_sentences = testing_dataset.get_saved_vectors("test", testing_dataset.dataset_type, vectors_filename)

        if len(testing_saved_vectors) == 0:
            testing_saved_vectors, testing_saved_labels, testing_saved_sentences, testing_saved_tokenized_sentences = testing_dataset.create_vectors("test", testing_dataset.dataset_type, vectors_filename)

        testing_left_input, testing_mid_input, testing_right_input = testing_dataset.format_sentences_tri_input_plus(testing_saved_tokenized_sentences)
        testing_lda_left_input, testing_lda_mid_input, testing_lda_right_input = testing_dataset.format_sentences_tri_input(testing_saved_vectors)

        # get class weight
        neg, pos = np.bincount(labels.flatten())
        initial_bias = np.log([pos/neg])

        total=len(labels)
        weight_for_0 = (1 / neg)*(total)/2.0
        weight_for_1 = (1 / pos)*(total)/2.0

        class_weight = {0: weight_for_0, 1: weight_for_1}
        print("class weight", class_weight)

        # create checkpoint path
        checkpoint_filepath = '{}/models/LDABERT2/simple/{}-{}-{}-pct-{}-aug_{}/finetune/checkpoint.ckpt'.format(
                                config.root_path,
                                dataset.dataset_type,
                                len(saved_sentences),
                                dataset.pct_data,
                                dataset.augment_pct,
                                random_hash)

        if step == "frozen":
            # create checkpoint path
            checkpoint_filepath = '{}/models/LDABERT2/simple/{}-{}-{}-pct-{}-aug_{}/finetune/checkpoint.ckpt'.format(
                                    config.root_path,
                                    dataset.dataset_type,
                                    len(saved_sentences),
                                    dataset.pct_data,
                                    dataset.augment_pct,
                                    random_hash)

        # # continue training
        # checkpoint_filepath = '{}/models/LDABERT2/simple/wiki-40561-1-pct-0-aug_jA6HL/finetune/checkpoint.ckpt'.format(config.root_path)

        print(checkpoint_filepath)

        # get callbacks ready.
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath,
            save_weights_only=True,
            monitor='val_accuracy',
            save_best_only=True,
            mode="auto",
            save_freq="epoch")

        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            verbose=1,
            patience=10,
            mode='max',
            restore_best_weights=True)

        callbacks = [
        #     early_stopping,
            model_checkpoint_callback
        ]

        # compiling model
        print("compiling the model...")
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                      loss=tf.keras.losses.BinaryCrossentropy(),
                      metrics=[
                          keras.metrics.BinaryAccuracy(name='accuracy')
                      ])

        try:
            model.load_weights(checkpoint_filepath)
            print("model loaded.")
        except:
            print("No checkpoint available.")

        print(f"starting the {step} training process...")
        history = model.fit([
                                left_input, mid_input, right_input,
                                lda_left_input, lda_mid_input, lda_right_input
                            ],
                            tf.convert_to_tensor(saved_labels),
                            validation_data=([
                                testing_left_input, testing_mid_input, testing_right_input,
                                testing_lda_left_input, testing_lda_mid_input, testing_lda_right_input
                            ], tf.convert_to_tensor(testing_saved_labels)),
                            epochs=finetuning_epochs if step == "finetune" else epochs,
                            # validation_split=0.25,
                            batch_size=BATCH_SIZE,
                            verbose=1,
                            class_weight=class_weight,
                            callbacks=callbacks)

        # assigning history to experiment object for saving.
        experiment["history"] = history.history
        experiment["hash"] = random_hash

        print("saving results...")
        save_results(experiment)

params: {'bert_type': 'ldabert', 'dataset_type': 'fiction', 'final_dropout': 0.5, 'dense_neurons': 128, 'max_sentence_length': 128, 'gamma': 15, 'pct_data': 1, 'augment_pct': 1, 'bert_trainable': True, 'epochs': 50}
initializing model...


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


initializing dataset...
processing dataset...
artificial segments False
artificial segments False
initializing testing dataset...
processing testing dataset...
artificial segments False
class weight {0: 0.5224410295628898, 1: 11.640308839190629}
/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/models/LDABERT2/simple/fiction-43721-1-pct-0-aug_DRh4S/finetune/checkpoint.ckpt
compiling the model...
No checkpoint available.
starting the finetune training process...
Epoch 1/5


...

2733/2733 [==============================] - 666s 229ms/step - loss: 0.6139 - accuracy: 0.6917 - val_loss: 0.6779 - val_accuracy: 0.6956
Epoch 2/5
2733/2733 [==============================] - 622s 228ms/step - loss: 0.5820 - accuracy: 0.6985 - val_loss: 0.6441 - val_accuracy: 0.7008
Epoch 3/5
2733/2733 [==============================] - 622s 228ms/step - loss: 0.5686 - accuracy: 0.7030 - val_loss: 0.6384 - val_accuracy: 0.7051
Epoch 4/5
2733/2733 [==============================] - 625s 229ms/step - loss: 0.5599 - accuracy: 0.7062 - val_loss: 0.6751 - val_accuracy: 0.7068
Epoch 5/5
2733/2733 [==============================] - 622s 228ms/step - loss: 0.5471 - accuracy: 0.7070 - val_loss: 0.5966 - val_accuracy: 0.7096
saving results...
initializing model...


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


initializing dataset...
processing dataset...
artificial segments False
artificial segments False
initializing testing dataset...
processing testing dataset...
artificial segments False
class weight {0: 0.5224410295628898, 1: 11.640308839190629}
/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/models/LDABERT2/simple/fiction-43721-1-pct-0-aug_DRh4S/finetune/checkpoint.ckpt
compiling the model...
model loaded.
starting the frozen training process...
Epoch 1/50


2733/2733 [==============================] - ETA: 0s - loss: 0.5313 - accuracy: 0.7468

2733/2733 [==============================] - 659s 229ms/step - loss: 0.5313 - accuracy: 0.7468 - val_loss: 0.6445 - val_accuracy: 0.7331
Epoch 2/50
2733/2733 [==============================] - 623s 228ms/step - loss: 0.5210 - accuracy: 0.7311 - val_loss: 0.4650 - val_accuracy: 0.7391
Epoch 3/50
2733/2733 [==============================] - 620s 227ms/step - loss: 0.5146 - accuracy: 0.7434 - val_loss: 0.5279 - val_accuracy: 0.7450
Epoch 4/50
2733/2733 [==============================] - 623s 228ms/step - loss: 0.5001 - accuracy: 0.7471 - val_loss: 0.4605 - val_accuracy: 0.7496
Epoch 5/50
2733/2733 [==============================] - 626s 229ms/step - loss: 0.4967 - accuracy: 0.7524 - val_loss: 0.6087 - val_accuracy: 0.7516
Epoch 6/50
2733/2733 [==============================] - 620s 227ms/step - loss: 0.4885 - accuracy: 0.7514 - val_loss: 0.6595 - val_accuracy: 0.7502
Epoch 7/50
2733/2733 [==============================] - 620s 227ms/step - loss: 0.4781 - accuracy: 0.7495 - val_loss: 0.494

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()